In [24]:
!pip install torch

In [25]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn

In [26]:
data_dir = 'dataset_Seminar5'
num_clients = 10
client_data = {}

# Load each client’s training data
for i in range(1, num_clients + 1):
    X_path = os.path.join(data_dir, f'client_datasets/client_{i}_features.csv')
    y_path = os.path.join(data_dir, f'client_datasets/client_{i}_labels.csv')
    
    X = pd.read_csv(X_path, header=None).values  # shape: (num_samples, 270)
    y = pd.read_csv(y_path, header=None).values.flatten()  # shape: (num_samples,)
    
    client_data[i] = {'X': X, 'y': y}

# Load test data
X_test = pd.read_csv(os.path.join(data_dir, 'test_features.csv'), header=None).values
y_test = pd.read_csv(os.path.join(data_dir, 'test_labels.csv'), header=None).values.flatten()

print(X.shape)  # shape: (num_samples, 270)
print(y.shape)  # shape: (num_samples,)
print(X_test.shape)  # shape: (num_samples, 270)
print(y_test.shape)  # shape: (num_samples,)

(64, 270)
(64,)
(500, 270)
(500,)


Chosen architecture: CNN (Ressim())

In [27]:
for client_id in client_data:
    client_data[client_id]['y'] -= 1  # Now ranges 0-11

y_test -= 1  # Also adjust test labels

In [28]:
class ResSim(nn.Module):
    """
    A simplified ResNet-style network with residual connections and sequential blocks.
    
    Adapted for CSI input (flattened to 3x30x3).

    Architecture:
        - Two residual blocks: Conv → ReLU → Conv + skip
        - Each followed by MaxPool
        - Fully connected classifier

    Args:
        num_classes (int): Number of output classes.
    """
    def __init__(self, num_classes=12): # 12 different pose classes
        super(ResSim, self).__init__()

        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1)
        )
        self.shortcut1 = nn.Conv2d(3, 64, kernel_size=1)  # aligns input channels

        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1)
        )
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 1)
        self.fc = nn.Linear(64 * 7 * 4, num_classes)

    def forward(self, x):
        x = x.view(-1, 3, 30, 3)  # reshape input vector (270,) → (3, 30, 3)

        # First residual connection
        residual = self.shortcut1(x)
        x = self.block1(x)
        x = self.relu(x + residual)
        x = self.pool(x) # (64, 15, 3)

        # Second residual connection (no need for shortcut: same shape)
        residual = x
        x = self.block2(x)
        x = self.relu(x + residual)
        x = self.pool(x) # (64, 7, 3)

        x = x.view(x.size(0), -1) # flatten
        return self.fc(x)

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PoseClassifierCNN(nn.Module):
    def __init__(self, num_classes=12):
        super(PoseClassifierCNN, self).__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),  # (3, 30, 3) → (16, 30, 3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=(2, 1)),  # (16, 15, 3)
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # (32, 15, 3)
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(2, 1))  # (32, 7, 3)
        )
        
        self.fc = nn.Sequential(
            nn.Flatten(),                    # (32 * 7 * 3)
            nn.Linear(32 * 7 * 3, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)     # output logits for 12 classes
        )

    def forward(self, x):
        # Reshape input from (batch_size, 270) → (batch_size, 3, 30, 3)
        x = x.view(-1, 3, 30, 3)  # match channel-first format
        x = self.cnn(x)
        x = self.fc(x)
        return x

In [30]:
# Federated Learning Parameters
NUM_CLIENTS = 10
CLIENTS_PER_ROUND = 5
FL_ROUNDS = 30

# Local training parameters
LOCAL_EPOCHS = 5
BATCH_SIZE = 32
LEARNING_RATE = 0.001


In [31]:
class PoseClassifierFC(nn.Module):
    def __init__(self, num_classes=12):
        super(PoseClassifierFC, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(270, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )
    
    def forward(self, x):
        return self.net(x)


In [32]:
def local_train(model, X, y, epochs=LOCAL_EPOCHS, lr=LEARNING_RATE, batch_size=BATCH_SIZE):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    dataset = torch.utils.data.TensorDataset(torch.FloatTensor(X), torch.LongTensor(y))
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for _ in range(epochs):
        for batch_X, batch_y in loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    return model.state_dict(), len(dataset)


In [33]:
# Step 1: Train the model federatedly
global_model = federated_training(client_data, num_rounds=10, clients_per_round=5)

# Step 2: Evaluate the global model
evaluate(global_model, X_test, y_test)



--- FL ROUND 1 ---

--- FL ROUND 2 ---

--- FL ROUND 3 ---

--- FL ROUND 4 ---

--- FL ROUND 5 ---

--- FL ROUND 6 ---

--- FL ROUND 7 ---

--- FL ROUND 8 ---

--- FL ROUND 9 ---

--- FL ROUND 10 ---
Test Accuracy: 22.20%


In [34]:
import random
def local_train(model, X, y, epochs=LOCAL_EPOCHS, lr=LEARNING_RATE, batch_size=BATCH_SIZE):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    dataset = torch.utils.data.TensorDataset(torch.FloatTensor(X), torch.LongTensor(y))
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for _ in range(epochs):
        for batch_X, batch_y in loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    return model.state_dict(), len(dataset)

def fedavg(state_dicts, data_sizes):
    global_model = state_dicts[0].copy()
    total_size = sum(data_sizes)

    for key in global_model.keys():
        global_model[key] = sum(
            state_dicts[i][key] * (data_sizes[i] / total_size)
            for i in range(len(state_dicts))
        )
    return global_model

def federated_training(client_data, rounds=FL_ROUNDS, clients_per_round=CLIENTS_PER_ROUND):
    global_model = PoseClassifierFC()  # or PoseClassifierCNN()
    global_state = global_model.state_dict()

    for round_num in range(rounds):
        print(f"--- FL ROUND {round_num + 1} ---")
        selected = random.sample(list(client_data.keys()), clients_per_round)

        local_states = []
        local_sizes = []

        for cid in selected:
            local_model = PoseClassifierFC()
            local_model.load_state_dict(global_state)
            X = client_data[cid]['X']
            y = client_data[cid]['y']
            state, size = local_train(local_model, X, y)
            local_states.append(state)
            local_sizes.append(size)

        global_state = fedavg(local_states, local_sizes)
        global_model.load_state_dict(global_state)

    return global_model

def evaluate(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X_test)
        y_tensor = torch.LongTensor(y_test)
        preds = model(X_tensor).argmax(dim=1)
        acc = (preds == y_tensor).float().mean().item()
    print(f"\n🎯 Test Accuracy: {acc * 100:.2f}%")


In [35]:
# Train global model
global_model = federated_training(client_data)

# Evaluate on the shared test set
evaluate(global_model, X_test, y_test)


--- FL ROUND 1 ---
--- FL ROUND 2 ---
--- FL ROUND 3 ---
--- FL ROUND 4 ---
--- FL ROUND 5 ---
--- FL ROUND 6 ---
--- FL ROUND 7 ---
--- FL ROUND 8 ---
--- FL ROUND 9 ---
--- FL ROUND 10 ---
--- FL ROUND 11 ---
--- FL ROUND 12 ---
--- FL ROUND 13 ---
--- FL ROUND 14 ---
--- FL ROUND 15 ---
--- FL ROUND 16 ---
--- FL ROUND 17 ---
--- FL ROUND 18 ---
--- FL ROUND 19 ---
--- FL ROUND 20 ---
--- FL ROUND 21 ---
--- FL ROUND 22 ---
--- FL ROUND 23 ---
--- FL ROUND 24 ---
--- FL ROUND 25 ---
--- FL ROUND 26 ---
--- FL ROUND 27 ---
--- FL ROUND 28 ---
--- FL ROUND 29 ---
--- FL ROUND 30 ---

🎯 Test Accuracy: 3.40%


In [36]:
for i, client in client_data.items():
    print(f"Client {i} labels:", np.unique(client['y']))

Client 1 labels: [ 0  3  4  5  6  8 10]
Client 2 labels: [1 6]
Client 3 labels: [ 1  3  7  8  9 10]
Client 4 labels: [0 5 8]
Client 5 labels: [2 5 7]
Client 6 labels: [ 4  5 11]
Client 7 labels: [ 0  3  4  7  8  9 10 11]
Client 8 labels: [ 4  5  8 10]
Client 9 labels: [ 0  1  5  7  8 10 11]
Client 10 labels: [1 4 6]


In [37]:
def federated_training(client_data, test_data, model_cls, num_rounds=10, frac=0.5, local_epochs=1):
    global_model = model_cls()
    test_X, test_y = test_data

    for round in range(num_rounds):
        print(f"--- Round {round+1} ---")

        # 1. Select random subset of clients
        selected_clients = random.sample(list(client_data.keys()), int(frac * len(client_data)))

        local_weights = []
        local_sizes = []

        # 2. Each client trains on its local data
        for client_id in selected_clients:
            data = client_data[client_id]
            local_model = model_cls()
            local_model.load_state_dict(global_model.state_dict())

            updated_weights = local_train(local_model, data['X'], data['y'], epochs=local_epochs)
            local_weights.append(updated_weights)
            local_sizes.append(len(data['y']))

        # 3. Aggregate updates with FedAvg
        averaged_weights = fed_avg(local_weights, local_sizes)
        global_model.load_state_dict(averaged_weights)

        # 4. Evaluate on test set
        acc = evaluate(global_model, test_X, test_y)
        print(f"Test Accuracy after round {round+1}: {acc:.2f}%")

    return global_model

In [38]:
final_model = federated_training(
    client_data=client_data,
    test_data=(X_test, y_test),
    model_cls=PoseClassifierCNN,
    num_rounds=10,
    frac=0.5,         # 50% of clients per round
    local_epochs=10
)

--- Round 1 ---


AttributeError: 'tuple' object has no attribute 'keys'

In [ ]:
from typing import Tuple


def create_dataloader(X: torch.Tensor, y: torch.Tensor, batch_size=32) -> torch.utils.data.DataLoader:
    """Converts client data to a DataLoader."""
    dataset = torch.utils.data.TensorDataset(X, y)
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

def evaluate(
    model: nn.Module,
    X_test: torch.Tensor,
    y_test: torch.Tensor,
    criterion: nn.Module,
    device: torch.device,
) -> Tuple[float, float]:
    """Evaluates model on test data."""
    model.eval()
    test_loader = create_dataloader(X_test, y_test, batch_size=64)
    total_loss, correct = 0.0, 0
    
    with torch.inference_mode():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            total_loss += criterion(outputs, y).item()
            correct += (outputs.argmax(1) == y).sum().item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / len(y_test)
    return avg_loss, accuracy